# Assignment: MTA Arrival Clock

In the "MTA Arrival Clock" case study, we have created an alternative design to visualize the arrival time information that looks at the following:

<div><img src="https://user-images.githubusercontent.com/3606672/246465520-5b7cad68-4722-4192-ad69-d0916c8f9cea.svg" width=400/></div>

As noted in the case study, it has the following drawbacks:

* Only trains arriving within the next 10 min are displayed.
* When two or more trains have the same arrival time, the subway line symbols and corresponding annotations will overlap.

## Task:

In this assignment, your task is to come up with a new design that addresses at least one of the two aforementioned drawbacks. You can either use the existing dot plot design or come up with a complete new design. The requirements of the design is the following:

* The dimension of the visualization is fixed (833x200).
* The design should make it easy to extract the arrival time information for each of the upcoming trains.
* The arrival time information should be legible from afar.
* Be mindful about utilizing the available spaces.

Please use this notebook for this assignment.

In [1]:
import altair as alt
import pandas as pd
import numpy as np
import warnings
warnings.filterwarnings("ignore", category=FutureWarning)


df = pd.DataFrame({
    "line": ["F", "F", "E", "M", "K", "R", "X", "S"],
    "arrival_time": [3, 6, 6, 2, 10, 22, 37, 15],
    "destination": ["Jamaica-179 St", "Kings Hwy", "Jamaica-179 St", "Forest Hills-71 Av", "8 Avenue Dyker Heights", "BayRidge Av", "Nice Destination", "Staten Island"]})
line_color = pd.DataFrame({
    "line": ["E", "F", "M", "K", "R", "X", "S"],
    "color": ["#0039a6", "#ff6319", "#ff6319", "#0039a6", "yellow", "green", "yellow"]
})

In [2]:
# Your code here...



# NEW


In [3]:
df = df.merge(line_color, on="line", how="left")

In [4]:
df

,line,arrival_time,destination,color
0,F,3,Jamaica-179 St,#ff6319
1,F,6,Kings Hwy,#ff6319
2,E,6,Jamaica-179 St,#0039a6
3,M,2,Forest Hills-71 Av,#ff6319
4,K,10,8 Avenue Dyker Heights,#0039a6
5,R,22,BayRidge Av,yellow
6,X,37,Nice Destination,green
7,S,15,Staten Island,yellow


In [5]:
bg_df = pd.DataFrame({
    "x": np.arange(11)
})

# Main canvas (background)
main_canvas = alt.Chart(bg_df).mark_rect(
    color='white'
).properties(
    width=866,
    height=200
)


In [6]:
# Function to select train with least arrival time and create its section
def select_train_with_least_arrival_time(df):
    selected_train = df.loc[df['arrival_time'].idxmin()]

    t_train_line = selected_train['line']
    t_arrival_time = selected_train['arrival_time']
    t_destination = selected_train['destination']
    t_color = selected_train['color']  # Get the color from the merged DataFrame

    # Truncate destination if it's too long for the section width
    CHAR_WIDTH = 10  # Approximate width of each character in pixels
    CANVAS_WIDTH = 866
    CANVAS_HEIGHT = 200
    SECTION_WIDTH = 266
    # SECTION_WIDTH = 200
    MAX_CHARS = 17

    def truncate_text(text, max_chars):
        if len(text) > max_chars:
            return text[:max_chars - 3] + '...'  # Truncate and add '...'
        return text

    t_destination_truncated = truncate_text(t_destination, MAX_CHARS)

    # First section (left aligned rectangle)
    first_section = alt.Chart(bg_df).mark_rect(
        color='#e8e8e8',
        width=SECTION_WIDTH,
        align='left',
        x=0,
        cornerRadius=20,
        # cornerRadiusTopLeft=0,
        # cornerRadiusBottomLeft=0,
        stroke="black",
        strokeWidth=2
    ).properties(
        width=CANVAS_WIDTH,
        height=CANVAS_HEIGHT
    )

    # Train line circle (dynamic based on selected train and its color)
    train_line = alt.Chart(pd.DataFrame({
        'x': [0],
        'line': [t_train_line]
    })).mark_circle(
        size=6969,
        color=t_color  # Use dynamic color from merged DataFrame
    ).encode(
        x=alt.value(48),
        y=alt.value(60)
    )

    # Train line text (dynamic)
    line_text = alt.Chart(pd.DataFrame({
        'x': [0],
        'line': [t_train_line]
    })).mark_text(
        color='black',
        fontSize=50,
        font='Helvetica',
        fontWeight='bold'
    ).encode(
        x=alt.value(48),
        y=alt.value(60),
        text='line'
    )

    # Destination text (dynamic with truncation)
    destination_text = alt.Chart(pd.DataFrame({
        'x': [0],
        'destination': [t_destination_truncated]
    })).mark_text(
        color='black',
        fontSize=22,
        font='Helvetica',
        align='left',
        fontWeight='bold'
    ).encode(
        x=alt.value(95),
        y=alt.value(60),
        text='destination'
    )

    # Arrival time text (dynamic)
    time_text1 = alt.Chart(pd.DataFrame({
        'x': [0],
        'time': [str(t_arrival_time)]
    })).mark_text(
        color='black',
        fontSize=75,
        font='Helvetica',
        fontWeight='bold'
    ).encode(
        x=alt.value(95),
        y=alt.value(150),
        text='time'
    )

    # "min" label for time (static)
    time_text2 = alt.Chart(pd.DataFrame({
        'x': [0],
        'time': ['min']
    })).mark_text(
        color='black',
        fontSize=25,
        font='Helvetica',
        fontWeight='bold'
    ).encode(
        x=alt.value(140),
        y=alt.value(150),
        text='time'
    )

    # Combine all elements into one visualization for first section
    first_section_content = train_line + line_text + destination_text + time_text1 + time_text2

    print(t_color)
    # print(section_width)

    return first_section + first_section_content



In [7]:
first_train = select_train_with_least_arrival_time(df)
first_train_visualization = main_canvas + first_train
first_train_visualization

#ff6319


alt.LayerChart(...)

In [8]:
# # Function to create remaining sections for other trains
# def create_remaining_sections(df):
#     # Exclude the train with the least arrival time
#     min_arrival_time = df['arrival_time'].min()
#     remaining_trains_df = df[df['arrival_time'] != min_arrival_time].sort_values(by="arrival_time")

#     point_sections = []
#     previous_arrival_time = min_arrival_time  # Start with the first train's arrival time

#     # Iterate over remaining trains, starting from SPLIT_AT_POINT
#     for i, row in enumerate(remaining_trains_df.itertuples(), start=SPLIT_AT_POINT):

#         # Calculate width for each point after first section
#         point_width = CANVAS_WIDTH / (TOTAL_POINTS - 1)

#         # If current train has a different arrival time than previous one, darken background progressively
#         if row.arrival_time != previous_arrival_time:
#             darkness_level = int(200 - ((i - SPLIT_AT_POINT) * (800 - 150) / (TOTAL_POINTS - SPLIT_AT_POINT)))
#             hex_color = f'#{darkness_level:02x}{darkness_level:02x}{darkness_level:02x}'
#             previous_arrival_time = row.arrival_time
#         else:
#             hex_color = f'#{darkness_level:02x}{darkness_level:02x}{darkness_level:02x}'  # Same color as before

#         # Create section rectangle with dynamic background color
#         point_section_rect = alt.Chart(bg_df).mark_rect(
#             color=hex_color,
#             width=point_width,
#             align="left",
#             x=i * point_width,
#             cornerRadius=20,
#         ).properties(width=866, height=200)

#         # Add dynamic content inside each section
#         point_section_content = (
#             # Train line circle
#             alt.Chart(pd.DataFrame({"x": [i], "line": [row.line]}))
#             .mark_circle(size=6969, color="#FDB813")
#             .encode(x=alt.value(i * point_width + 43), y=alt.value(60))
#             +
#             # Train line text
#             alt.Chart(pd.DataFrame({"x": [i], "line": [row.line]}))
#             .mark_text(color="black", fontSize=50, font="Helvetica", fontWeight="bold")
#             .encode(x=alt.value(i * point_width + 43), y=alt.value(60), text="line")
#             +
#             # Destination text
#             alt.Chart(pd.DataFrame({"x": [i], "destination": [truncate_text(row.destination, MAX_CHARS)]}))
#             .mark_text(color="black", fontSize=10, font="Helvetica", align="center", fontWeight="bold")
#             .encode(x=alt.value(i * point_width + 42), y=alt.value(10), text="destination")
#             +
#             # Arrival time
#             alt.Chart(pd.DataFrame({"x": [i], "time": [str(row.arrival_time)]}))
#             .mark_text(color="black", fontSize=75, font="Helvetica", fontWeight="bold")
#             .encode(x=alt.value(i * point_width + 40), y=alt.value(150), text="time")
#         )

#         # Append this section to list of sections
#         point_sections.append(point_section_rect + point_section_content)

#     # Combine all sections into a single layer
#     all_point_sections_layered = alt.layer(*point_sections)

#     return all_point_sections_layered



In [9]:
# # Function to create remaining sections for other trains
# def create_remaining_sections(df):
#     # Exclude the train with the least arrival time
#     min_arrival_time = df['arrival_time'].min()
#     remaining_trains_df = df[df['arrival_time'] != min_arrival_time].sort_values(by='arrival_time')

#     point_sections = []
#     previous_arrival_time = min_arrival_time  # Start with the first train's arrival time

#     # Calculate the width for each point after the first section, adjusting as needed
#     # point_width = (CANVAS_WIDTH / (TOTAL_POINTS - 1)) * 1
#     point_width = 120

#     # Iterate over remaining trains, starting from SPLIT_AT_POINT
#     for i, row in enumerate(remaining_trains_df.itertuples(), start=SPLIT_AT_POINT):
#         # If current train has a different arrival time than previous one, darken background progressively
#         if row.arrival_time != previous_arrival_time:
#             darkness_level = int(200 - ((i - SPLIT_AT_POINT) * (350 - 150) / (TOTAL_POINTS - SPLIT_AT_POINT)))
#             hex_color = f'#{darkness_level:02x}{darkness_level:02x}{darkness_level:02x}'
#             previous_arrival_time = row.arrival_time
#         else:
#             hex_color = f'#{darkness_level:02x}{darkness_level:02x}{darkness_level:02x}'  # Same color as before

#         # Correct x position considering adjusted width
#         x_position = (i - SPLIT_AT_POINT) * point_width + section_width  # Adjusted to start after first section

#         # Create section rectangle with dynamic background color and adjusted width
#         point_section_rect = alt.Chart(bg_df).mark_rect(
#             color=hex_color,
#             width=point_width,
#             align="left",
#             x=x_position,  # Adjust for widths and correct starting position
#             cornerRadius=20,
#         ).properties(width=866, height=200)

#         # Add dynamic content inside each section
#         point_section_content = (
#             # Train line circle
#             alt.Chart(pd.DataFrame({"x": [i], "line": [row.line]}))
#             .mark_circle(size=6969, color=row.color)  # Use color from merged DataFrame
#             .encode(x=alt.value(x_position + 43), y=alt.value(60))
#             +
#             # Train line text
#             alt.Chart(pd.DataFrame({"x": [i], "line": [row.line]}))
#             .mark_text(color="white", fontSize=50, font="Helvetica", fontWeight="bold")
#             .encode(x=alt.value(x_position + 43), y=alt.value(60), text="line")
#             +
#             # Destination text
#             alt.Chart(pd.DataFrame({"x": [i], "destination": [truncate_text(row.destination, MAX_CHARS)]}))
#             .mark_text(color="black", fontSize=10, font="Helvetica", align="center", fontWeight="bold")
#             .encode(x=alt.value(x_position + 42), y=alt.value(10), text="destination")
#             +
#             # Arrival time
#             alt.Chart(pd.DataFrame({"x": [i], "time": [str(row.arrival_time)]}))
#             .mark_text(color="black", fontSize=75, font="Helvetica", fontWeight="bold")
#             .encode(x=alt.value(x_position + 40), y=alt.value(150), text="time")
#             +
#             # "min" label for time
#             alt.Chart(pd.DataFrame({"x": [i], "time": ['min']}))
#             .mark_text(color="black", fontSize=25, font="Helvetica", fontWeight="bold")
#             .encode(x=alt.value(x_position + 90), y=alt.value(150), text="time")  # Adjust x position slightly to align next to time
#         )

#         # Append this section to list of sections
#         point_sections.append(point_section_rect + point_section_content)

#     # Combine all sections into a single layer
#     all_point_sections_layered = alt.layer(*point_sections)

#     return all_point_sections_layered

In [10]:
# remaining_trains_visualization = create_remaining_sections(df)
# remaining_trains_visualization

In [11]:
# remaining_trains_visualization = create_remaining_sections(df)

# # Clip the remaining_trains_visualization to a max width of 866px
# clipped_remaining_visualization = clip_visualization_to_width(remaining_trains_visualization, max_width=500)
# clipped_remaining_visualization

In [12]:
# # Now combine both functions' results into one final visualization

# # First train visualization (from locked code)
# first_train_visualization = select_train_with_least_arrival_time(df)

# # Remaining trains visualization
# remaining_trains_visualization = create_remaining_sections(df)



In [13]:
# SPLIT_AT_POINT = 266

In [14]:
# # Function to create remaining sections for the next 5 trains
# def create_remaining_sections(df):
#     # Exclude the train with the least arrival time
#     min_arrival_time = df['arrival_time'].min()
#     remaining_trains_df = df[df['arrival_time'] != min_arrival_time].sort_values(by='arrival_time')

#     # Limit to only the next 5 trains
#     remaining_trains_df = remaining_trains_df.head(5)

#     point_sections = []
#     previous_arrival_time = min_arrival_time  # Start with the first train's arrival time

#     # Set a fixed width for each section (you can adjust this as needed)
#     point_width = 120

#     # Iterate over remaining trains, starting from SPLIT_AT_POINT
#     for i, row in enumerate(remaining_trains_df.itertuples(), start=SPLIT_AT_POINT):
#         # If current train has a different arrival time than previous one, darken background progressively
#         if row.arrival_time != previous_arrival_time:
#             darkness_level = int(200 - ((i - SPLIT_AT_POINT) * (350 - 150) / (TOTAL_POINTS - SPLIT_AT_POINT)))
#             hex_color = f'#{darkness_level:02x}{darkness_level:02x}{darkness_level:02x}'
#             previous_arrival_time = row.arrival_time
#         else:
#             hex_color = f'#{darkness_level:02x}{darkness_level:02x}{darkness_level:02x}'  # Same color as before

#         # Correct x position considering adjusted width
#         x_position = (i - SPLIT_AT_POINT) * point_width + section_width  # Adjusted to start after first section

#         # Create section rectangle with dynamic background color and adjusted width
#         point_section_rect = alt.Chart(bg_df).mark_rect(
#             color=hex_color,
#             width=point_width,
#             align="left",
#             x=x_position,  # Adjust for widths and correct starting position
#             cornerRadius=20,
#         ).properties(width=866, height=200)

#         # Add dynamic content inside each section
#         point_section_content = (
#             # Train line circle
#             alt.Chart(pd.DataFrame({"x": [i], "line": [row.line]}))
#             .mark_circle(size=6969, color=row.color)  # Use color from merged DataFrame
#             .encode(x=alt.value(x_position + 43), y=alt.value(60))
#             +
#             # Train line text
#             alt.Chart(pd.DataFrame({"x": [i], "line": [row.line]}))
#             .mark_text(color="white", fontSize=50, font="Helvetica", fontWeight="bold")
#             .encode(x=alt.value(x_position + 43), y=alt.value(60), text="line")
#             +
#             # Destination text
#             alt.Chart(pd.DataFrame({"x": [i], "destination": [truncate_text(row.destination, MAX_CHARS)]}))
#             .mark_text(color="black", fontSize=10, font="Helvetica", align="center", fontWeight="bold")
#             .encode(x=alt.value(x_position + 42), y=alt.value(10), text="destination")
#             +
#             # Arrival time
#             alt.Chart(pd.DataFrame({"x": [i], "time": [str(row.arrival_time)]}))
#             .mark_text(color="black", fontSize=75, font="Helvetica", fontWeight="bold")
#             .encode(x=alt.value(x_position + 40), y=alt.value(150), text="time")
#             +
#             # "min" label for time
#             alt.Chart(pd.DataFrame({"x": [i], "time": ['min']}))
#             .mark_text(color="black", fontSize=25, font="Helvetica", fontWeight="bold")
#             .encode(x=alt.value(x_position + 90), y=alt.value(150), text="time")  # Adjust x position slightly to align next to time
#         )

#         # Append this section to list of sections
#         point_sections.append(point_section_rect + point_section_content)

#     # Combine all sections into a single layer
#     all_point_sections_layered = alt.layer(*point_sections)

#     return all_point_sections_layered

In [15]:
# remaining_trains_visualization = create_remaining_sections(df)
# remaining_trains_visualization

# from PIL import Image

# # Load the image (replace 'image.jpg' with your actual image file path)
# image = Image.open('image.jpg')

# # Define crop dimensions (left, upper, right, lower)
# # Adjust these values based on how much you want to crop from each side
# crop_box = (0, 0, 500, 150)  # Example dimensions

# # Crop the image
# cropped_image = image.crop(crop_box)

# # Save the cropped image to a new file
# cropped_image.save('cropped_image.jpg')

# # Optionally display the cropped image
# cropped_image.show()

In [16]:
# # Function to create remaining sections for other trains
# def create_remaining_sections(df):
#     # Exclude the train with the least arrival time
#     min_arrival_time = df['arrival_time'].min()
#     remaining_trains_df = df[df['arrival_time'] != min_arrival_time].sort_values(by='arrival_time')

#     point_sections = []
#     previous_arrival_time = min_arrival_time  # Start with the first train's arrival time

#     # Calculate the width for each point after the first section, adjusting as needed
#     point_width = (CANVAS_WIDTH / (TOTAL_POINTS - 1)) * 1.5

#     # Iterate over remaining trains, starting from SPLIT_AT_POINT
#     for i, row in enumerate(remaining_trains_df.itertuples(), start=SPLIT_AT_POINT):
#         # If current train has a different arrival time than previous one, darken background progressively
#         if row.arrival_time != previous_arrival_time:
#             darkness_level = int(200 - ((i - SPLIT_AT_POINT) * (400 - 150) / (TOTAL_POINTS - SPLIT_AT_POINT)))
#             hex_color = f'#{darkness_level:02x}{darkness_level:02x}{darkness_level:02x}'
#             previous_arrival_time = row.arrival_time
#         else:
#             hex_color = f'#{darkness_level:02x}{darkness_level:02x}{darkness_level:02x}'  # Same color as before

#         # Correct x position considering adjusted width
#         x_position = (i - SPLIT_AT_POINT) * point_width + section_width  # Adjusted to start after first section

#         # Create section rectangle with dynamic background color and adjusted width
#         point_section_rect = alt.Chart(bg_df).mark_rect(
#             color=hex_color,
#             width=point_width,
#             align="left",
#             x=x_position,  # Adjust for widths and correct starting position
#             cornerRadius=20,
#         ).properties(width=866, height=200)

#         # Add dynamic content inside each section
#         point_section_content = (
#             # Train line circle
#             alt.Chart(pd.DataFrame({"x": [i], "line": [row.line]}))
#             .mark_circle(size=6969, color=row.color)  # Use color from merged DataFrame
#             .encode(x=alt.value(x_position + 43), y=alt.value(60))
#             +
#             # Train line text
#             alt.Chart(pd.DataFrame({"x": [i], "line": [row.line]}))
#             .mark_text(color="black", fontSize=50, font="Helvetica", fontWeight="bold")
#             .encode(x=alt.value(x_position + 43), y=alt.value(60), text="line")
#             +
#             # Destination text
#             alt.Chart(pd.DataFrame({"x": [i], "destination": [truncate_text(row.destination, MAX_CHARS)]}))
#             .mark_text(color="black", fontSize=10, font="Helvetica", align="center", fontWeight="bold")
#             .encode(x=alt.value(x_position + 42), y=alt.value(10), text="destination")
#             +
#             # Arrival time
#             alt.Chart(pd.DataFrame({"x": [i], "time": [str(row.arrival_time)]}))
#             .mark_text(color="black", fontSize=75, font="Helvetica", fontWeight="bold")
#             .encode(x=alt.value(x_position + 40), y=alt.value(150), text="time")
#             +
#             # "min" label for time
#             alt.Chart(pd.DataFrame({"x": [i], "time": ['min']}))
#             .mark_text(color="black", fontSize=25, font="Helvetica", fontWeight="bold")
#             .encode(x=alt.value(x_position + 90), y=alt.value(150), text="time")  # Adjust x position slightly to align next to time
#         )

#         # Append this section to list of sections
#         point_sections.append(point_section_rect + point_section_content)

#     # Combine all sections into a single layer
#     all_point_sections_layered = alt.layer(*point_sections)

#     # Clip visualization to a maximum width of 866px by limiting x-axis range and setting properties
#     return all_point_sections_layered.encode(
#         x=alt.X(scale=alt.Scale(domain=[0, CANVAS_WIDTH]))  # Limit x-axis range to clip extra content
#     ).properties(width=500, height=200)


In [17]:

def truncate_text(text, max_chars):
  if len(text) > max_chars:
    return text[:max_chars - 3] + '...'  # Truncate and add '...'
  return text

# t_destination_truncated = truncate_text(t_destination, MAX_CHARS)


In [25]:
# Function to create remaining sections for the next 5 trains
def create_remaining_sections(df):
    # Exclude the train with the least arrival time
    min_arrival_time = df['arrival_time'].min()
    remaining_trains_df = df[df['arrival_time'] != min_arrival_time].sort_values(by='arrival_time')

    # Limit to only the next 5 trains
    remaining_trains_df = remaining_trains_df.head(5)

    point_sections = []
    previous_arrival_time = min_arrival_time  # Start with the first train's arrival time

    # Define SPLIT_AT_POINT and TOTAL_POINTS
    SPLIT_AT_POINT = 1  # Assuming sections start after the first section (which is already created)
    TOTAL_POINTS = len(remaining_trains_df) + SPLIT_AT_POINT  # Total points include first section + remaining trains

    # Set a fixed width for each section (you can adjust this as needed)
    point_width = 120
    section_width = 266  # This is the width of the first section

    # Define truncate_text function here or import it if it's defined elsewhere
    def truncate_text(text, max_chars):
        if len(text) > max_chars:
            return text[:max_chars - 3] + '...'  # Truncate and add '...'
        return text

    MAX_CHARS = 12  # Maximum characters for destination text

    # Iterate over remaining trains, starting from SPLIT_AT_POINT
    for i, row in enumerate(remaining_trains_df.itertuples(), start=SPLIT_AT_POINT):
        # If current train has a different arrival time than previous one, darken background progressively
        if row.arrival_time != previous_arrival_time:
            darkness_level = int(200 - ((i - SPLIT_AT_POINT) * (269 - 150) / (TOTAL_POINTS - SPLIT_AT_POINT)))
            hex_color = f'#{darkness_level:02x}{darkness_level:02x}{darkness_level:02x}'
            previous_arrival_time = row.arrival_time
        else:
            hex_color = f'#{darkness_level:02x}{darkness_level:02x}{darkness_level:02x}'  # Same color as before

        # Correct x position considering adjusted width
        x_position = (i - SPLIT_AT_POINT) * point_width + section_width  # Adjusted to start after first section

        # Create section rectangle with dynamic background color and adjusted width
        point_section_rect = alt.Chart(bg_df).mark_rect(
            color=hex_color,
            width=point_width,
            align="left",
            x=x_position,  # Adjust for widths and correct starting position
            cornerRadius=20,
            cornerRadiusTopLeft=0,
            cornerRadiusBottomLeft=0,
            stroke="black",
            strokeWidth=0.3
        ).properties(width=866, height=200)

        # Add dynamic content inside each section
        point_section_content = (
            # Train line circle
            alt.Chart(pd.DataFrame({"x": [i], "line": [row.line]}))
            .mark_circle(size=6969, color=row.color)  # Use color from merged DataFrame
            .encode(x=alt.value(x_position + 60), y=alt.value(70))
            +
            # Train line text
            alt.Chart(pd.DataFrame({"x": [i], "line": [row.line]}))
            .mark_text(color="white", fontSize=50, font="Helvetica", fontWeight="bold")
            .encode(x=alt.value(x_position + 60), y=alt.value(70), text="line")
            +
            # Destination text
            alt.Chart(pd.DataFrame({"x": [i], "destination": [truncate_text(row.destination, MAX_CHARS)]}))
            .mark_text(color="black", fontSize=14, font="Helvetica", align="center", fontWeight="bold")
            .encode(x=alt.value(x_position + 60), y=alt.value(19), text="destination")
            +
            # Arrival time
            alt.Chart(pd.DataFrame({"x": [i], "time": [str(row.arrival_time)]}))
            .mark_text(color="black", fontSize=69, font="Helvetica", fontWeight="bold")
            .encode(x=alt.value(x_position + 60), y=alt.value(150), text="time")
            +
            # "min" label for time
            alt.Chart(pd.DataFrame({"x": [i], "time": ['min']}))
            .mark_text(color="black", fontSize=18, font="Helvetica", fontWeight="bold")
            .encode(x=alt.value(x_position + 60), y=alt.value(184), text="time")  # Adjust x position slightly to align next to time
        )

        # Append this section to list of sections
        point_sections.append(point_section_rect + point_section_content)

    # Combine all sections into a single layer
    all_point_sections_layered = alt.layer(*point_sections)

    return all_point_sections_layered


remaining_trains_visualization = create_remaining_sections(df)
remaining_trains_visualization


alt.LayerChart(...)

In [26]:
# Final visualization by layering both first and remaining sections
final_visualization_with_all_sections = main_canvas + first_train_visualization + remaining_trains_visualization

# Display final visualization
final_visualization_with_all_sections

alt.LayerChart(...)

In [27]:
print("GG")

GG



# Design Elaboration

- The design has a **prominent tile** displaying details for the next arriving train, providing essential information for subway riders.
- Additional tiles represent upcoming trains in sequence. Each tile showcases the train's arrival time in a large, bold font, making it **easily readable from a distance**, along with the circular subway line logo for quick recognition.
- Background color shades gradually darken to indicate the increasing wait time for each subsequent train, offering riders a visual cue on how soon each train will arrive.

# Design Advantages

- High Visibility from distance of two main elements, that is, train logo and time for arrival.
- **Solves Challenge 2**: The design effectively handles trains arriving at the same time by keeping the background color consistent for these trains, making it easier to interpret simultaneous arrivals.
- **Solves Challenge 1**: The display can show up to six upcoming trains, including those arriving in more than 10 minutes. As each tile grows darker in color, it visually reinforces the idea of longer wait times for trains further down the timeline.
- The layout avoids clutter by **managing long texts representing destination**  effectively.
	- Improvement: For longer train names, we can implement a scrolling animation to prevent truncation and ensure readability.

# Design Limitations

- For trains with the same line symbol but different destinations, riders may need to approach the screen to read the detailed text.
- **Limited Future Display**: If six trains are arriving within a short span (e.g., 10 minutes), information about trains arriving later may be hidden.
	- Improvement: Adding a scrolling or paging function to access trains arriving beyond the initial six displayed.
- **Color Interpretation**: The subtle gradation in grey shades might be misinterpreted, especially by riders in a hurry. For instance, the fifth tile could represent either a train arriving soon (e.g., in 5 minutes) or much later (e.g., 25 minutes), depending on the timing of preceding trains.